In [1]:
# from twitter_scraper import get_tweets
import os
import sys
sys.path.insert(0, os.path.abspath('../../Modules/twitter-scraper/twitter_scraper/modules'))

from tweets import get_tweets

In [2]:
# Collect tweets
trump_tweets = []

for tweet in get_tweets('realDonaldTrump', pages=200):
    trump_tweets.append(tweet['text'])

In [27]:
trump_tweets[:5]

['Too bad we didn’t have the G-7 here. I offered to pick up the entire cost, would have saved at least $35,000,000 for the USA. Best location. Very stupid people thought I would gain. Wrong! Looking at Camp David. Will announce soon https://twitter.com/trumpgolf/status/1197943099781201922\xa0…',
 'Polls have now turned very strongly against Impeachment, especially in swing states. 75% to 25%. Thank you!',
 'Nancy Pelosi, Adam Schiff, AOC and the rest of the Democrats are not getting important legislation done, hence, the Do Nothing Democrats. USMCA, National Defense Authorization Act, Gun Safety, Prescription Drug Prices, & Infrastructure are dead in the water because of the Dems!',
 'Navy Seal Eddie Gallagher will be on @foxandfriends this morning at 7:30 A.M. Have no fear, all will end well for everyone!',
 'Pushed hard to have Apple build in USA https://twitter.com/tim_cook/status/1197141315064086530\xa0…']

## Tokenize

In [82]:
import spacy

# NLP Model
nlp = spacy.load("en_core_web_md")

In [83]:
def tokenize(text):
    # Tokenize
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if (token.is_stop != True) and (token.is_punct != True)]

    return tokens

In [87]:
import pandas as pd

# Send to series
trump_tweets_series = pd.Series(trump_tweets)

In [89]:
# Tokenize
tokens = trump_tweets_series.apply(tokenize)

In [99]:
# Clean up tokens
for token in tokens:
    if " " in token:
        token.remove(" ")
    if "  " in token:
        token.remove("  ")
    if "’s" in token:
        token.remove("’s")
    for token_bit in token:
        if token_bit.startswith("https"):
            token.remove(token_bit)

## Vectorize

In [102]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [103]:
# Instantiate Vectorizer
tfidf = TfidfVectorizer(stop_words = 'english')

In [105]:
texts = []

for token in tokens:
    text = " ".join(token)
    texts.append(text)

In [106]:
# Create a vocab and get word counts per doc
sparse = tfidf.fit_transform(texts)

In [107]:
# send to df
tfidf_dtm = pd.DataFrame(sparse.todense(), columns = tfidf.get_feature_names())

## Model

In [112]:
vocab = sorted(set(tfidf_dtm.columns))

In [125]:
# Batch size
BATCH_SIZE = 2

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [126]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
      ])
    return model

In [127]:
import tensorflow as tf

model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [128]:
# def loss(labels, logits):
#     return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [136]:
from tensorflow.keras.losses import sparse_categorical_crossentropy
model.compile(optimizer='adam', loss=sparse_categorical_crossentropy)

In [132]:
EPOCHS=10

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [137]:
history = model.fit(tfidf_dtm, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10


InvalidArgumentError: logits and labels must have the same first dimension, got logits shape [4462,2231] and labels shape [1]
	 [[{{node loss_2/dense_1_loss/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits}}]]